# Credentials and initialize conneciton

Follow https://developers.google.com/earth-engine/reference/Quickstart#before-you-begin to get a secret key

In [1]:
import json

KEY = 'my-secret-key.json'
with open(KEY, 'r') as f:
    data = json.load(f)
    SERVICE_ACCOUNT = data['client_email']
    PROJECT = data['project_id']

In [2]:
import ee

ee_creds = ee.ServiceAccountCredentials(SERVICE_ACCOUNT, KEY)
ee.Initialize(ee_creds)

# NDVI around coordinate

In [ ]:
import geemap

# --- Add NDVI and date index bands before masking ---
def add_ndvi_and_index_before_mask(image, index):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    index_band = ee.Image.constant(index).rename('index').toFloat()
    return image.addBands([ndvi, index_band])

# --- Mask Sentinel-2 bad data ---
def mask_s2_bad_data(image):
    scl = image.select('SCL')
    bad_classes = [0, 1, 2, 3, 8, 9, 10]
    mask = scl.remap(bad_classes, [0]*len(bad_classes), 1).eq(1)
    return image.updateMask(mask)

# --- Pull area function ---
def pull_area(geom, time_start, time_end):
    # Sentinel-2 image collection
    collection = (
        ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 40))
        .filterBounds(geom)
        .filterDate(time_start, time_end)
        .sort('system:time_start', True)
    )

    images_list = collection.toList(collection.size())
    dates = []
    processed_images = []

    # Process each image
    for i in range(images_list.size().getInfo()):
        img = ee.Image(images_list.get(i))
        date_str = ee.Date(img.get('system:time_start')).format('YYYY-MM-dd').getInfo()
        dates.append(date_str)
        img_with_bands = add_ndvi_and_index_before_mask(img, i)
        img_masked = mask_s2_bad_data(img_with_bands)
        processed_images.append(img_masked)

    # Rebuild collection
    indexed_collection = ee.ImageCollection(processed_images)
    filled = indexed_collection.qualityMosaic('index').clip(geom)

    # Extract bands
    ndvi = filled.select('NDVI')
    index = filled.select('index')
    newest_raw = collection.sort('system:time_start', False).first().clip(geom)
    rgb_raw = newest_raw.select(['B4', 'B3', 'B2']).divide(10000)
    rgb_filled = filled.select(['B4', 'B3', 'B2']).divide(10000)

    # Dynamic World grass mosaic
    dw_collection = (
        ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')
        .filterBounds(geom)
        .filterDate(time_start, time_end)
    )

    dw_image = dw_collection.qualityMosaic('grass').clip(geom)
    dw_grass_mask = dw_image.select('grass').gt(0.3)

    # Create vector outline
    dw_outline = dw_grass_mask.reduceToVectors(
        geometry=geom,
        scale=50,
        geometryType='polygon',
        eightConnected=False,
        labelProperty='grass',
        reducer=ee.Reducer.countEvery()
    )

    dw_outline = (
        dw_outline
        .filter(ee.Filter.gt('grass', 0))
        .map(lambda f: f.buffer(30).simplify(30))
        .map(lambda f: f.set('area', f.geometry().area()))
        .filter(ee.Filter.gt('area', 100000))  # ~1 ha
    )

    # Visualization params
    ndvi_vis = {'min': 0, 'max': 1.0, 'palette': ['red', 'yellow', 'green']}
    index_vis = {'min': 0, 'max': len(dates) - 1, 'palette': ['black', 'white']}
    rgb_vis = {'min': 0.0, 'max': 0.3}
    dw_params = {'min': 0, 'max': 1, 'palette': ['white', 'green']}

    # Create and populate map
    Map = geemap.Map(center=[-2.75, 38.55], zoom=12)
    Map.addLayer(ndvi, ndvi_vis, 'Cloud-Filled NDVI')
    Map.addLayer(index, index_vis, 'Image Recency Index (black=newest)')
    Map.addLayer(rgb_raw, rgb_vis, 'RGB Composite (Raw)')
    Map.addLayer(rgb_filled, rgb_vis, 'RGB Composite (Filtered)')
    Map.addLayer(dw_image.select('grass'), dw_params, 'Dynamic World (Grass)')
    Map.addLayer(dw_outline, {'color': 'blue'}, 'Dynamic World (Grass Outline)')

    # Print mapping
    print("Index to Date mapping:")
    for idx, d in enumerate(dates):
        print(f"Index {idx}: {d}")

    return Map, ndvi, dw_outline


# Example usage
geom = ee.Geometry.BBox(38.4, -2.9, 38.7, -2.6)
time_start = '2024-05-10'
time_end = '2024-06-10'
Map, ndvi, dw_outline = pull_area(geom, time_start, time_end)

# Show map (for notebooks)
Map


Index to Date mapping:
Index 0: 2024-05-14
Index 1: 2024-05-14
Index 2: 2024-05-19
Index 3: 2024-05-19
Index 4: 2024-05-24
Index 5: 2024-05-24
Index 6: 2024-05-29
Index 7: 2024-05-29
Index 8: 2024-06-03
Index 9: 2024-06-08
Index 10: 2024-06-08


Map(center=[-2.75, 38.55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [51]:
# Create a binary mask image from dw_outline
dw_mask = ee.Image(0).byte().paint(featureCollection=dw_outline, color=1)
masked_ndvi = ndvi.updateMask(dw_mask)

# Compute mean NDVI under the mask
mean_ndvi_dict = masked_ndvi.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=geom,
    scale=10,
    maxPixels=1e9
)
mean_ndvi = mean_ndvi_dict.get('NDVI').getInfo()
print(f"\nAverage NDVI in Dynamic World 'grass' area: {mean_ndvi:.3f}")

# Visualize the masked NDVI
masked_ndvi_vis = {'min': 0, 'max': 1.0, 'palette': ['red', 'yellow', 'green']}
Map.addLayer(masked_ndvi, masked_ndvi_vis, 'NDVI (Masked to Grass)')



Average NDVI in Dynamic World 'grass' area: 0.263
